In [1]:

!pip install firebase-admin
!pip install google-cloud-storage
!pip install python-dotenv
!pip install pandas

In [2]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

In [3]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }
  

In [4]:
import firebase_admin
from firebase_admin import credentials, storage
   
   # Method 1: Direct file path
cred = credentials.Certificate('D:/project/Vietnamese-Restaurant-Service-Chatbot/chatbot-service-ba6d2-firebase-adminsdk-ui9sr-f9154790d8.json')
firebase_admin.initialize_app(cred, {
    'storageBucket': 'chatbot-service-ba6d2.firebasestorage.app',  # Remove the gs:// prefix
    'databaseURL': 'https://chatbot-service-ba6d2-default-rtdb.firebaseio.com/'
})

In [5]:
bucket = storage.bucket()

In [6]:
image_folder_path = './products/images/'
products_collection = db.reference('products')
df = pd.read_json('D:/project/Vietnamese-Restaurant-Service-Chatbot/products/vietnamese_food_detail.jsonl', lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Pho Ga,Noodles,A delicious Vietnamese noodle soup with tender...,"[Chicken, Rice Noodles, Broth, Herbs]",6.5,4.8,pho_ga.jpg
1,Goi Cuon,Seafood,Fresh Vietnamese spring rolls filled with shri...,"[Shrimp, Rice Paper, Vermicelli, Herbs]",5.0,4.7,goi_cuon.jpg


In [12]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url
 
for index, row in df.iterrows():
    print(index, row['name'])
    
    # Correct way to build the image path
    image_path = os.path.join('d:/project/Vietnamese-Restaurant-Service-Chatbot/products/images', row['image_path'])
    
    image_url = upload_image(bucket,image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url']= image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)
    

0 Pho Ga
1 Goi Cuon
2 Hu Tieu Nam Vang
3 Tra Da
4 Nuoc Mia
5 Banh Xeo
6 Ca Phe Sua Da
7 Banh Mi Thit
8 Banh Trang Nuong
9 Xoi Ga
10 Bun Rieu
11 Goi Ga
12 Banh Pia
13 Bun Cha
14 Kem Xoi
15 Bun Dau Mam Tom
16 Bun Oc
17 Banh Beo
18 Nuoc Chanh
19 Nem Nuong
20 Banh Mi Cha Ca
21 Banh Khot
22 Sinh To Bo
23 Bun Thit Nuong
24 Com Ga
25 Lau Nam
26 Che Troi Nuoc
27 Sinh To Xoai
28 Com Tam
29 Che Ba Mau
30 Bo Nuong La Lot
